In [1]:
import pandas as pd
from pymongo import MongoClient
from sklearn.preprocessing import LabelEncoder
import ast
from sklearn.preprocessing import StandardScaler


In [22]:
import pandas as pd

# Define base path
base_path = r"C:/Users/acer/Desktop/due_deligence/data_risk"

# Load market data
market_data = pd.read_csv(f"{base_path}\\market_data.csv")

# Load news data
news_data = pd.read_csv(f"{base_path}\\news_data.csv")

# Quick peek
print(market_data.head())
print(news_data.head())

# Convert to DataFrames
market_df = pd.DataFrame(market_data)
news_df = pd.DataFrame(news_data)


            id symbol      name  current_price     market_cap  \
0      bitcoin    btc   Bitcoin   97781.000000  1942716505416   
1     ethereum    eth  Ethereum    2765.400000   334591411004   
2       ripple    xrp       XRP       2.480000   143481543884   
3       tether   usdt    Tether       0.999995   141952534193   
4  binancecoin    bnb       BNB     700.950000   102259685465   

   market_cap_rank  fully_diluted_valuation  total_volume  high_24h  \
0                1            1942719935467  4.989618e+10  98098.00   
1                2             334591411004  2.400374e+10   2782.27   
2                3             248365405217  4.822054e+09      2.49   
3                4             141952534193  4.632736e+10      1.00   
4                5             102259685465  2.390417e+09    703.18   

        low_24h  ...  total_supply    max_supply        ath  \
0  94120.000000  ...  1.982338e+07  2.100000e+07  108786.00   
1   2565.920000  ...  1.205431e+08           NaN    4878

In [25]:
def get_crypto_data(symbol, market_df, news_df):
    # Get market info
    market_info = market_df[market_df['symbol'].str.lower() == symbol.lower()]
    if market_info.empty:
        return None, None

    # Get news mentioning the currency
    news_related = news_df[
        news_df['trading_pair_or_currencies'].str.lower().str.contains(symbol.lower(), na=False)
    ].sort_values(by="time", ascending=False).head(5)  # limit to recent 5

    return market_info.to_dict(orient='records')[0], news_related.to_dict(orient='records')


In [30]:
symbol = 'bnb'  # Example symbol
market_info, news_related = get_crypto_data(symbol, market_data, news_data)

# Print the results
print("Market Info:", market_info)
print("News Related:", news_related)


Market Info: {'id': 'binancecoin', 'symbol': 'bnb', 'name': 'BNB', 'current_price': 700.95, 'market_cap': 102259685465, 'market_cap_rank': 5, 'fully_diluted_valuation': 102259685465, 'total_volume': 2390417337.0, 'high_24h': 703.18, 'low_24h': 627.79, 'price_change_24h': 58.58, 'price_change_percentage_24h': 9.1198, 'market_cap_change_24h': 8578463228.0, 'market_cap_change_percentage_24h': 9.15708, 'circulating_supply': 145887575.79, 'total_supply': 145887575.79, 'max_supply': 200000000.0, 'ath': 788.84, 'ath_change_percentage': -10.86365, 'ath_date': '2024-12-04T10:35:25.220Z', 'atl': 0.0398177, 'atl_change_percentage': 1765814.27166, 'atl_date': '2017-10-19T00:00:00.000Z', 'roi': nan, 'last_updated': '2025-02-12T22:46:24.704Z'}
News Related: [{'title': 'BNB Surpasses 660 USDT with a 2.94% Decrease in 24 Hours', 'content': 'Binance Market Data, BNB crossed the 660 USDT benchmark and is now trading at 660.099976 USDT, with a 2.94% decrease in 24 hours.\nTrading Pair: BNB\nPrice Change:

In [ ]:
%pip install ollama

In [35]:

import ollama
import json

def ask_llama_about_risk(symbol, market_info, news_related):
    prompt = f"""
You are a financial risk analyst. A user is asking about the RISK of investing in the cryptocurrency '{symbol}'.

Here is the market data:
{json.dumps(market_info, indent=2)}

Here are recent news articles:
{json.dumps(news_related, indent=2)}

Based on this information, tell me:
- Is this cryptocurrency risky?
- Why or why not?
- Summarize the decision in a single line at the end: RISK LEVEL = [LOW/MEDIUM/HIGH]
"""

    response = ollama.chat(
        model='llama3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Return the response content
    return response['message']['content']


In [36]:
symbol = "btc"  # or input("Enter crypto symbol: ")
market_info, news_info = get_crypto_data(symbol, market_data, news_data)

if market_info:
    response = ask_llama_about_risk(symbol, market_info, news_info)
    print(response)
else:
    print("Symbol not found in market data.")


As a financial risk analyst, I would evaluate the risk of investing in Bitcoin (BTC) based on the provided market data and recent news articles.

The current price of BTC is $97,481, which is relatively high compared to its historical lows. The market capitalization of $194 billion is significant, indicating a large number of investors have already invested in the cryptocurrency.

Recent news articles suggest that there are some concerns about the direction of the market. For instance:

* Grayscale's GBTC experienced net outflows, while Mini BTC saw net inflows, indicating potential shifts in investor sentiment.
* Ray Dalio discussed the depreciation of currencies against assets like gold and Bitcoin, which may indicate a broader trend.
* A dormant Bitcoin address containing 300 BTC was activated after over a decade, potentially signaling increased activity.

However, there are no major red flags or significant negative news articles that would indicate high risk. The price change over

In [ ]:
%pip install flask



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


: 

In [1]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Example: Assuming you have a set of true labels and predicted labels for the risk levels (HIGH, MEDIUM, LOW)
true_labels = ["HIGH", "LOW", "MEDIUM", "MEDIUM", "HIGH"]  # Actual labels
predicted_labels = ["HIGH", "LOW", "MEDIUM", "HIGH", "LOW"]  # Model predictions

# Calculate Precision, Recall, and F1-Score
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")


Precision: 0.7
Recall: 0.6
F1-Score: 0.6


In [2]:
!pip install nltk



[notice] A new release of pip available: 22.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from nltk.translate.bleu_score import sentence_bleu

# Example: Reference text and generated text
reference = [["The", "price", "of", "BTC", "is", "rising", "quickly."]]
generated = ["The", "price", "of", "BTC", "is", "increasing", "rapidly."]

# Calculate BLEU score (using 1-gram and 2-gram)
score = sentence_bleu(reference, generated)
print(f"BLEU Score: {score}")


BLEU Score: 0.6147881529512643
